# Gaussian Function

---

## $$f(x) = \frac{1}{ \sqrt {2 \pi \sigma^2 } } e^{- \frac{(x-\mu)^2}{2 \sigma^2}}$$

In [33]:
import numpy as np
from math import *

In [31]:
# utilizing NumPy
def gaussian_func(x, mu, s2):
    return 1/np.sqrt(2 * np.pi * s2) * np.exp(-.5 * (x-mu)**2 / s2)

In [32]:
gaussian_func(x=8, mu=10, s_2=4)

0.12098536225957168

In [35]:
# utilizing math
def f(x1, m, sigma2):
    return 1/sqrt(2 * pi * sigma2) * exp(-0.5 * (x1-m)**2 / sigma2)

In [36]:
f(x1=10, m=10, sigma2=4)

0.19947114020071635

---

# `Univariate` Kalman Filter

## `Bayes' Rule` Measurement Update

$\mu$ = prior mean

$\sigma^2$ = prior variance

$v$ (aka $\mu2 $) = mean "likelihood"

$r^2$ (aka $\sigma2 $) = variance "likelihood"

$$\text{New Mean}$$
## $$\mu^` = \frac{r^2 \mu + \sigma^2 v}{r^2 + \sigma^2}$$

<br>
<br>

$$\text{New Variance}$$
## $$\sigma^2` = \frac{1}{\frac{1}{r^2} + \frac{1}{\sigma^2}}$$

#### Measurement update

In [6]:
def measurement_update(mu1, s1, mu2, s2):
    new_mean = (s2 * mu1 + s1 * mu2) / (s2 + s1)
    new_var = 1 / (1/s2 + 1/s1)
    return [new_mean, new_var]


In [7]:
assert measurement_update(10., 8., 13., 2.) == [12.4, 1.6]

#### Motion update

In [1]:
def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return [new_mean, new_var]


In [10]:
assert predict(10., 8., 12., 3.) == [22.0, 11.0]

---

In [18]:
measurements = [5., 6., 7., 9., 10.]
motion = [1., 1., 2., 1., 1.]
measurement_sig = 4.
motion_sig = 2.
mu = 0.
sig = 10000.


for n in range(len(measurements)):
    [mu, sig] = measurement_update(mu, sig, measurements[n], measurement_sig)
    print(f"Update:  {[mu, sig]}")
    [mu, sig] = predict(mu, sig, motion[n], motion_sig)
    print(f"Predict: {[mu, sig]}" + "\n")


Update:  [4.998000799680128, 3.9984006397441023]
Predict: [5.998000799680128, 5.998400639744102]

Update:  [5.999200191953932, 2.399744061425258]
Predict: [6.999200191953932, 4.399744061425258]

Update:  [6.999619127420922, 2.0951800575117594]
Predict: [8.999619127420921, 4.09518005751176]

Update:  [8.999811802788143, 2.0235152416216957]
Predict: [9.999811802788143, 4.023515241621696]

Update:  [9.999906177177365, 2.0058615808441944]
Predict: [10.999906177177365, 4.005861580844194]



<br>

---

# `Multivariate` Kalman Filter

* below code adapted from _[Udacity's AI for Robotics](https://www.udacity.com/course/artificial-intelligence-for-robotics--cs373)_ course

In [5]:
# Write a function 'kalman_filter' that implements a multi-
# dimensional Kalman Filter for the example given

from math import *

class matrix:
    
    # implements basic operations of a matrix class
    
    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0
    
    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            raise (ValueError, "Invalid size of matrix")
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]
    
    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
            raise (ValueError, "Invalid size of matrix")
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1
    
    def show(self):
        for i in range(self.dimx):
            print(self.value[i])
        print(' ')
    
    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise (ValueError, "Matrices must be of equal dimensions to add")
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res
    
    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise (ValueError, "Matrices must be of equal dimensions to subtract")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res
    
    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
            raise (ValueError, "Matrices must be m*n and n*p to multiply")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res
    
    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res
    
    # Thanks to Ernesto P. Adorio for use of Cholesky and CholeskyInverse functions
    
    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                    raise (ValueError, "Matrix not positive-definite")
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
        return res
    
    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res
    
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res
    
    def __repr__(self):
        return repr(self.value)


In [8]:
########################################

# Implement the filter function below

def kalman_filter(x, P):
    for n in range(len(measurements)):
        # measurement update step
        Z = matrix([[measurements[n]]])
        y = Z - (H * x)  # Error
        S = H * P * H.transpose() + R
        K = P * H.transpose() * S.inverse()  # Kalman Gain
        x = x + (K * y)
        
        P = (I - (K*H)) * P
        
        # prediction step
        x = (F * x) + u
        P = F * P * F.transpose()
        
    return x,P

############################################
### use the code below to test your filter!
############################################

measurements = [1, 2, 3]

x = matrix([[0.], [0.]]) # initial state (location and velocity)
P = matrix([[1000., 0.], [0., 1000.]]) # initial uncertainty
u = matrix([[0.], [0.]]) # external motion
F = matrix([[1., 1.], [0, 1.]]) # next state function
H = matrix([[1., 0.]]) # measurement function
R = matrix([[1.]]) # measurement uncertainty
I = matrix([[1., 0.], [0., 1.]]) # identity matrix

soln = kalman_filter(x, P)
# output should be:
# x: [[3.9996664447958645], [0.9999998335552873]]
# P: [[2.3318904241194827, 0.9991676099921091], [0.9991676099921067, 0.49950058263974184]]

print(f"x: {soln[0]}")
print(f"P: {soln[1]}")

x: [[3.9996664447958645], [0.9999998335552873]]
P: [[2.3318904241194827, 0.9991676099921091], [0.9991676099921067, 0.49950058263974184]]
